# 三、编写网络爬虫

## 3.1 遍历单个域名

以“维基百科的六度分隔理论”为例，我们进行单个域名的遍历。

In [1]:
from urllib.request import urlopen
from bs4 import BeautifulSoup

html = urlopen('http://en.wikipedia.org/wiki/Kevin_Bacon')
bs = BeautifulSoup(html, 'html.parser')
for link in bs.find_all('a'):
    if 'href' in link.attrs:
        print(link.attrs['href'])

HTTPError: HTTP Error 502: Bad Gateway

观察一下我们需要的链接元素的特征，做一点简单的过滤：

- 他们都在id是bodyContent的div标签里
- URL不包含冒号
- URL都以/wiki/开头

In [2]:
from urllib.request import urlopen
from bs4 import BeautifulSoup
import re

html = urlopen('http://en.wikipedia.org/wiki/Kevin_Bacon')
bs = BeautifulSoup(html, 'html.parser')
for link in bs.find('div', {'id': 'bodyContent'}).find_all('a', href=re.compile('^(/wiki/)((?!:).)*$')):
    if 'href' in link.attrs:
        print(link.attrs['href'])

HTTPError: HTTP Error 502: Bad Gateway

让代码更像一个完整的程序：

- 一个函数getLinks
- 一个主函数

In [ ]:
import datetime
import random
import re

from urllib.request import urlopen
from bs4 import BeautifulSoup

random.seed(datetime.datetime.now())
def getLinks(articleUrl: str)-> list:
    html = urlopen('http://en.wikipedia.org{}'.format(articleUrl))
    bs = BeautifulSoup(html, 'html.parser')
    return bs.find('div', {'id': 'bodyContent'}).find_all('a', href=re.compile('^(/wiki/)((?!:).)*$'))

def main():
    links = getLinks('http://en.wikipedia.org/wiki/Kevin_Bacon')
    while len(links) > 0:
        newArticle = links[random.randint(0, len(links)-1)].attrs['href']
        print(newArticle)
        links = getLinks(newArticle)

## 3.2 抓取整个网站

遍历整个网站的作用：

1. 生成网站地图
2. 广泛收集数据

In [7]:
import re

from urllib.request import urlopen
from bs4 import BeautifulSoup

pages = set()
def getLinks(pageUrl):
    global pages
    html = urlopen('http://en.wikipedia.org{}'.format(pageUrl))
    bs = BeautifulSoup(html, 'html.parser')
    for link in bs.find_all('a', href=re.compile('^(/wiki/)')):
        if 'href' in link.attrs:
            if link.attrs['href'] not in pages:
                # We have encounter a nwe page
                newPage = link.attrs['href']
                print(newPage)
                pages.add(newPage)
                getLinks(newPage)

getLinks('')

BeautifulSoup
BeautifulStoneSoup
CData
CSS
Comment
Counter
DEFAULT_OUTPUT_ENCODING
Declaration
Doctype
FeatureNotFound
GuessedAtParserWarning
HTMLParserTreeBuilder
MarkupResemblesLocatorWarning
NavigableString
PYTHON_SPECIFIC_ENCODINGS
PageElement
ParserRejectedMarkup
ProcessingInstruction
ResultSet
Script
SoupStrainer
StopParsing
Stylesheet
Tag
TemplateString
UnicodeDammit
XMLParsedAsHTMLWarning


## 3.3 收集整个网站的数据

每个页面我们最好能做点什么，而不是提取链接就走：

- 提取所有标题
- 提取正文文本的第一段文字
- 提取编辑链接

In [ ]:
import re

from urllib.request import urlopen
from bs4 import BeautifulSoup

pages = set()
def getLinks(pageUrl):
    global pages
    html = urlopen('http://en.wikipedia.org{}'.format(pageUrl))
    bs = BeautifulSoup(html, 'html.parser')
    try:
        print(bs.h1.get_text())
        print(bs.find(id='mw-content-text'.find_all('p')[0]))
        print(bs.find(id='ca-edit').find('span')).find('a').attrs['href']
    except AttributeError:
        print("页面缺少一些属性！不过不用担心！")

    for link in bs.find_all('a', href=re.compile('^(/wiki/)')):
        if 'href' in link.attrs:
            if link.attrs['href'] not in pages:
                # We have encounter a nwe page
                newPage = link.attrs['href']
                print('-'*20)
                print(newPage)
                pages.add(newPage)
                getLinks(newPage)

getLinks('')

> 注意：重定向处理
>
> 使用requests库时，需要将允许重定向的标志设置为True：
>
> `r = requests.get('http://github.com', allow_redirects=True)`

## 3.3 在互联网上抓取

之前，我们只在网站内爬取，即忽略外链。但是现在，我们要跟着外链跳转！

> 注意：接下来的代码可以到达互联网的任何地方。

跳转外链之前，请问自己几个问题：

- 我要收集那些数据？数据收集可以通过抓取几个预定义的网站（永远是最简单的做法）完成吗？或者我的爬虫需要能够发现那些我可能不知道的网站吗？
- 当我的爬虫到达某个网站，它是立即顺着下一个出站链接跳到下一个新网站，还是在网站上停留一会儿，深入抓取网站的内容？
- 有没有我不想抓取的一类网站？我对非英文网站的内容感兴趣吗？
- 如果我的网络爬虫引起了某个网站管理员的怀疑，我如何避免承担法律责任？

In [ ]:
import re
import datetime
import random

from urllib.request import urlopen
from urllib.parse import urlparse
from bs4 import BeautifulSoup

pages = set()
random.seed(datetime.datetime.now())

# 获取页面中所有内链的列表
def getInternalLinks(bs: BeautifulSoup, includeUrl: str) -> list:
    includeUrl = '{}://{}'.format(urlparse(includeUrl).sheme, urlparse(includeUrl).netloc)
    internalLinks = []
    # 找出所有以“/”开头的链接
    for link in bs.find_all('a', href=re.compile('^(/|.*)' + includeUrl + ')')):
        if link.attrs['href'] is not None:
            if link.attrs['href'].startswith('/'):
                internalLinks.append(includeUrl + link.attrs['href'])
            else:
                internalLinks.append(link.attrs['href'])
    return internalLinks

# 获取页面中素有外链的列表
def getExternalLinks(bs: BeautifulSoup, excludeUrl: str) -> list:
    externalLinks = []
    # 找出所有以“http”或“www”开头且不包含当前URL的链接
    for link in bs.find_all('a', href=re.compile('^(http|www)((?!' + excludeUrl + ').)*$')):
        if link.attrs['href'] is not None:
            if link.attrs['href'] not in externalLinks:
                externalLinks.append(link.attrs['href'])
    return externalLinks

def getRandomExternalLink(startingPage: str) -> list:
    html = urlopen(startingPage)
    bs = BeautifulSoup(html, 'html.parser')
    externalLinks = getExternalLinks(bs, urlparse(startingPage).netloc)
    if len(externalLinks) == 0:
        print('No external links, looking around the site for one')
        domain = '{}://{}'.format(urlparse(startingPage).scheme, urlparse(startingPage).netloc)
        internalLinks = getInternalLinks(bs, domain)
        return getRandomExternalLink(internalLinks[random.randint(0, len(internalLinks)-1)])
    else:
        return externalLinks[random.randint(0, len(externalLinks)-1)]
    
def followExternalOnly(startingSite: str) -> None:
    externalLink = getRandomExternalLink(startingSite)
    print('Random external link is: {}'.format(externalLink))
    followExternalOnly(externalLink)

followExternalOnly('http://oreilly.com')

> 不要把上面的示例代码放入任何产品代码中，因为它缺乏必要的异常检测，因此爬虫的稳健性不足。
>
> 一种增强爬虫稳健性的方法：将其与Chapter1中介绍的处理网络连接异常的代码结合起来。这样，当出现HTTP错误或服务器异常时，代码就可以选择一个不同的URL。

任务分解成一个个的小函数可以方便的重构代码，以满足另一个抓取任务的需求。例如：现在的目标变成了抓取一个网站中所有外链并且逐一记录下来，可以增加以下函数：

In [ ]:
allExtLinks = set()
allIntLinks = set()

def getAllExternalLinks(siteUrl: str) -> None:
    html = urlopen(siteUrl)
    domain = '{}://{}'.format(urlparse(siteUrl).scheme, urlparse(siteUrl).netloc)
    bs = BeautifulSoup(html, 'html.parser')
    internalLinks = getInternalLinks(bs, domain)
    externalLinks = getExternalLinks(bs, domain)

    for link in externalLinks:
        if link not in allExtLinks:
            allExtLinks.add(link)
            print(link)

    for link in internalLinks:
        if link not in allIntLinks:
            allIntLinks.add(link)
            print(link)

allIntLinks.add('http://oreilly.com')
getAllExternalLinks('http://oreilly.com')

> 写代码之前拟个大纲或者画个流程图是个很好的编程习惯，这么做不仅可以为后期处理节省很多时间，更重要的是可以防止自己在爬虫变得越来越复杂时乱了方寸。